In [9]:
import pandas as pd
import numpy as np
from IPython.display import display
from sklearn.datasets import make_classification, make_regression
from ProQSAR.ModelDeveloper.model_developer import ModelDeveloper
#from ProQSAR.Uncertainty.conformal_predictor import ConformalPredictor
from typing import Iterable

def create_classification_data(
    n_samples=40, n_features=40, n_informative=10, random_state=42
    ):
    X, y = make_classification(n_samples=n_samples, n_features=n_features,
                                n_informative=n_informative, random_state=random_state)
    data = pd.DataFrame(X, columns=[f'Feature{i}' for i in range(1, n_features + 1)])
    data["ID"] = np.arange(n_samples)
    data["Activity"] = y
    return data

def create_regression_data(
    n_samples=40, n_features=40, n_informative=10, random_state=42
    ):
    X, y = make_regression(n_samples=n_samples, n_features=n_features, n_informative=n_informative, random_state=random_state)
    data = pd.DataFrame(X, columns=[f'Feature{i}' for i in range(1, n_features + 1)])
    data["ID"] = np.arange(n_samples)
    data["Activity"] = y
    return data

train_data = create_regression_data(random_state=42)
cal_data = create_regression_data(random_state=41)
test_data = create_regression_data(random_state=40)

In [10]:
import os
import pickle
import logging
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator
from sklearn.exceptions import NotFittedError
from typing import Optional, Union, Iterable
from ProQSAR.ModelDeveloper.model_developer_utils import _get_task_type
from ProQSAR.ModelDeveloper.model_developer import ModelDeveloper
from mapie.regression import MapieRegressor
from mapie.classification import MapieClassifier


class ConformalPredictor(BaseEstimator):
    """
    A class to perform conformal prediction for classification and regression models.
    """

    def __init__(
        self,
        model: Optional[Union[ModelDeveloper, BaseEstimator]] = None,
        activity_col: str = "activity",
        id_col: str = "id",
        n_jobs: int = -1,
        random_state: Optional[int] = 42,
        save_dir: Optional[str] = "Project/ConformalPredictor",
        deactivate: bool = False,
        **kwargs,
    ) -> None:
        """
        Initialize the ConformalPredictor class.

        Args:
            model (Union[ModelDeveloper, object]): The model to be used for conformal prediction.
            activity_col (str): The target variable column name.
            id_col (str): The identifier column name.
            save_dir (Optional[str]): Directory to save calibration and prediction results.
        """
        self.model = model
        self.activity_col = activity_col
        self.id_col = id_col
        self.n_jobs = n_jobs
        self.random_state = random_state
        self.save_dir = save_dir
        self.deactivate = deactivate
        self.cp_kwargs = kwargs
        self.task_type = None
        self.cp_kwargs = kwargs

    def fit(self, data: pd.DataFrame):

        if self.deactivate:
            logging.info("ConformalPredictor is deactivated. Skipping calibrate.")
            return self
        
        if isinstance(self.model, ModelDeveloper):
            self.model = self.model.model

        try:
            # get X & y
            X_data = data.drop(
                [self.activity_col, self.id_col], axis=1, errors="ignore"
            )
            y_data = data[self.activity_col]

            # get task_type
            self.task_type = _get_task_type(data, self.activity_col)

            if self.task_type == "C":
                self.cp = MapieClassifier()
            elif self.task_type == "R":
                self.cp = MapieRegressor()
            else:
                raise ValueError("Unsupported task type detected.")

            self.cp.set_params(
                estimator=self.model,
                n_jobs=self.n_jobs,
                random_state=self.random_state,
                **self.cp_kwargs,
            )
            self.cp.fit(X=X_data, y=y_data)

            if self.save_dir:
                os.makedirs(self.save_dir, exist_ok=True)
                with open(f"{self.save_dir}/conformal_predictor.pkl", "wb") as file:
                    pickle.dump(self, file)

            logging.info("Conformal predictor calibrated successfully.")
            return self

        except Exception as e:
            logging.error(f"Error during calibration: {e}")
            raise

    def predict(
        self,
        data: pd.DataFrame,
        alpha: Optional[Union[float, Iterable[float]]] = None,
    ) -> pd.DataFrame:
        """
        Generate conformal prediction intervals or sets.

        Args:
            data (pd.DataFrame): Dataset for making predictions.
            confidence (float): Confidence level for prediction intervals.
            **kwargs: Additional parameters for prediction.

        Returns:
            pd.DataFrame: DataFrame containing prediction results.
        """
        # check_is_fitted(self.predictor)
        if self.cp is None:
            raise NotFittedError(
                "ConformalPredictor is not calibrated yet. Call 'calibrate' before using this function."
            )
        try:
            X_data = data.drop(
                [self.activity_col, self.id_col], axis=1, errors="ignore"
            )
            y_pred = self.cp.predict(X=X_data, alpha=alpha)

            results_list = []
            if isinstance(alpha, float):
                alpha = [alpha]

            for i in range(len(X_data)):
                # Get ID
                sample_data = {self.id_col: data[self.id_col].iloc[i]}

                # Get actual value if available
                if self.activity_col in data.columns:
                    sample_data[self.activity_col] = data[self.activity_col].iloc[i]

                if alpha:
                    sample_data["Predicted value"] = y_pred[0][i]
                    sample_cal = y_pred[1][i, :, :]
                    for k, a in enumerate(alpha):
                        if self.task_type == "C":
                            class_labels = self.cp.classes_
                            set_indices = np.where(sample_cal[:, k])[0]
                            result = class_labels[set_indices]

                        elif self.task_type == "R":
                            result = np.round(sample_cal[:, k], decimals=3)

                        sample_data[f"Prediction Set (alpha={a})"] = result
                else:
                    sample_data["Predicted value"] = y_pred[i]

                results_list.append(sample_data)

            pred_result = pd.DataFrame(results_list)

            if self.save_dir:
                os.makedirs(self.save_dir, exist_ok=True)
                pred_result.to_csv(f"{self.save_dir}/conformal_pred_result.csv")

            logging.info("Prediction completed successfully.")
            return pred_result

        except Exception as e:
            logging.error(f"Error during prediction: {e}")
            raise

    def set_params(self, **kwargs):
        valid_keys = self.__dict__.keys()
        for key in kwargs:
            if key not in valid_keys:
                raise KeyError(f"'{key}' is not a valid attribute.")
        self.__dict__.update(**kwargs)

        return self


In [11]:
cp = ConformalPredictor(activity_col="Activity", id_col="ID", n_jobs=-1)
cp.fit(train_data)

ConformalPredictor(activity_col='Activity', id_col='ID')

In [21]:
cp.cp.conformity_scores_ = cp.cp.conformity_scores_.astype(np.float64)

In [22]:
cp.cp.conformity_scores_

array([ 33.68750405, 102.86792097,  72.96247946,  46.87192528,
        61.80296038, 155.85607703, 174.77341122,  52.93035736,
        11.30124547,   6.21413565,  59.26953605,  70.59488409,
        16.42249957,  27.94486494,  76.54139278, 106.32475609,
       127.29046668,  74.41266258,  10.08119214,  83.01468812,
        72.52865861,  69.93551808,  44.33147981,  49.81888035,
        47.08253907,   9.19907546,  11.36876741, 149.45458908,
       180.4704714 ,  92.48827881, 113.49975044,  49.53726435,
        71.07287514,  96.28032265,  87.08595989, 105.12810846,
        24.93281724,  25.32443548,  25.767469  ,   6.83656886])

In [23]:
pred = cp.predict(train_data, alpha=(0.3))
pred

,ID,Activity,Predicted value,Prediction Set (alpha=0.3)
0,0,-141.293986,-141.293986,"[-214.256, -36.166]"
1,1,-18.495373,-18.495373,"[-145.786, 54.033]"
2,2,-84.059004,-84.059004,"[-203.893, -7.518]"
3,3,102.989907,102.989907,"[0.122, 179.531]"
4,4,-73.086714,-73.086714,"[-169.367, 1.326]"
5,5,-176.370524,-176.370524,"[-248.899, -49.08]"
6,6,268.845890,268.845890,"[141.555, 345.387]"
7,7,-66.066175,-66.066175,"[-153.152, 39.062]"
8,8,28.543209,28.543209,"[-54.471, 121.031]"
9,9,52.653380,52.653380,"[-39.835, 135.668]"


In [95]:
a = [0.05, 1].format[]
a

SyntaxError: invalid decimal literal (3120431655.py, line 1)

In [60]:
for k, alpha in enumerate(b):
    print(k)
    print(alpha)

0
[0.05]


In [23]:
X_train = train_data.drop(
        ["Activity", "ID"], axis=1, errors="ignore"
    )
y_train = train_data["Activity"]

In [6]:
y_train

0     0
1     0
2     1
3     0
4     1
5     0
6     0
7     1
8     0
9     0
10    1
11    1
12    1
13    0
14    0
15    1
16    0
17    1
18    0
19    0
20    1
21    1
22    1
23    0
24    1
25    1
26    1
27    1
28    1
29    0
30    0
31    1
32    0
33    1
34    0
35    1
36    0
37    1
38    0
39    0
Name: Activity, dtype: int64

In [24]:
import os
import pickle
import logging
import pandas as pd
from sklearn.base import BaseEstimator
from sklearn.utils.validation import check_is_fitted
from sklearn.exceptions import NotFittedError
from typing import Optional, Union, List, Dict, Iterable
from ProQSAR.ModelDeveloper.model_developer_utils import _get_task_type
from ProQSAR.ModelDeveloper.model_developer_utils import _get_cv_strategy
from mapie.regression import MapieRegressor
from mapie.classification import MapieClassifier
from sklearn.svm import SVC, SVR

cp = MapieClassifier(estimator=SVC(probability=True), cv=5)
cp.fit(X=X_train, y=y_train)

MapieClassifier(cv=5, estimator=SVC(probability=True))

In [25]:
X_test = test_data.drop(
        ["Activity", "ID"], axis=1, errors="ignore"
    )

In [20]:
y_test = test_data["Activity"]
y_test

0     1
1     0
2     0
3     0
4     1
5     1
6     1
7     1
8     1
9     0
10    1
11    0
12    0
13    1
14    0
15    1
16    0
17    0
18    1
19    0
20    1
21    0
22    1
23    0
24    0
25    1
26    1
27    0
28    1
29    1
30    0
31    1
32    1
33    0
34    0
35    1
36    0
37    1
38    0
39    0
Name: Activity, dtype: int64

In [32]:
y_pred = cp.predict(X=X_test)
y_pred

array([0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1])

In [33]:
y_pred[0]

0

In [31]:
y_pred[1][1,:,:]

array([[ True,  True],
       [ True,  True]])

In [33]:
sample_cal = y_cal[1, :, :]
sample_cal

array([[ True,  True],
       [ True,  True]])

In [39]:
y_pred, y_int = cp.predict(X=X_test, alpha=[0.05, 0.1])
class_labels = cp.classes_

results_list = []
for i in range(len(X_test)):
    sample_int= y_int[i, :, :]
    sample_data = {
        "True Label": y_test[i],
        "Prediction": y_pred[i]
    }

    for k, alpha in enumerate([0.05, 0.1]):
        set_indices = np.where(sample_cal[:, 0])[0]
        set_labels = class_labels[set_indices]
        sample_data[f"Prediction Set (alpha={alpha})"] = set_labels 
    
    results_list.append(sample_data)

results_df = pd.DataFrame(results_list)


In [40]:
results_df

,True Label,Prediction,Prediction Set (alpha=0.05),Prediction Set (alpha=0.1)
0,1,0,"[0, 1]","[0, 1]"
1,0,1,"[0, 1]","[0, 1]"
2,0,1,"[0, 1]","[0, 1]"
3,0,1,"[0, 1]","[0, 1]"
4,1,0,"[0, 1]","[0, 1]"
5,1,1,"[0, 1]","[0, 1]"
6,1,1,"[0, 1]","[0, 1]"
7,1,0,"[0, 1]","[0, 1]"
8,1,0,"[0, 1]","[0, 1]"
9,0,0,"[0, 1]","[0, 1]"


In [22]:
cp.classes_

array([0, 1])

In [13]:
X_test = test_data.drop(
        ["Activity", "ID"], axis=1, errors="ignore"
    )

In [6]:
X_test

,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,Feature10,...,Feature31,Feature32,Feature33,Feature34,Feature35,Feature36,Feature37,Feature38,Feature39,Feature40
0,-0.881655,-1.489821,-0.593804,-6.506184,-2.536080,1.751379,1.335209,0.527204,-0.994012,0.927148,...,1.074952,-2.392715,0.257832,-1.560045,2.029329,0.539981,1.534535,-1.097862,-1.093619,0.068512
1,-0.092626,-2.812287,1.150590,-5.155610,0.419282,1.006032,0.734486,-0.937687,-1.630897,0.322001,...,1.161645,-0.091492,-0.343070,-0.898390,1.786529,0.608757,2.778813,0.208876,0.266279,0.686832
2,-0.387491,-4.995783,1.087250,-6.372869,0.320686,-0.149824,-1.398471,2.429047,0.938634,1.051764,...,-0.261454,-0.328138,0.963111,0.241168,0.576488,-0.237837,1.105235,1.599163,-2.291386,0.210546
3,-0.942033,-0.808456,-0.494331,3.396730,-0.518698,0.054583,-1.068674,-2.119399,0.387217,-2.788881,...,0.487762,-0.758368,1.926787,-1.069833,0.481143,-0.247781,-1.530513,-0.280747,0.795492,1.293821
4,1.475009,0.594325,-2.044336,2.515483,-0.605635,1.118840,-0.597636,0.939141,-2.385944,0.589067,...,-0.196423,-0.120029,0.717928,-0.396366,0.132269,1.637517,2.260704,0.760935,-0.167874,-1.040686
5,-0.580574,3.607257,0.899256,-1.985242,-2.851890,1.788404,1.194187,-0.199018,0.406053,-0.206630,...,-0.168720,-0.435643,-0.144467,-1.097297,0.444826,1.051993,-0.433779,0.495057,0.382398,-0.735407
6,-0.220221,1.620978,0.510497,-4.671659,-1.817591,-0.365788,0.103009,1.056934,-0.613828,1.277336,...,0.443642,-0.846812,-0.172899,-0.933716,1.624726,-0.693070,0.487316,2.170331,-1.681795,0.832244
7,-0.459617,-3.967689,0.575759,-4.900004,0.434052,0.130769,-0.246720,0.132560,0.769767,-0.182862,...,-0.667729,0.039864,0.121547,-0.766934,0.004194,0.398380,2.887782,0.216005,-0.806046,0.620686
8,1.404243,0.877563,0.299306,-6.885686,0.854555,-0.025810,0.179817,-0.832889,0.988677,-0.957021,...,1.729150,-0.162279,0.513972,3.478311,0.094241,-0.753455,-0.281375,0.153548,1.124123,-0.187502
9,1.908917,-2.802470,0.328684,1.257898,-6.503409,0.004297,0.171307,2.555880,-1.319554,0.718727,...,1.028155,-1.405039,1.361203,-3.835030,0.270497,2.345475,4.005916,1.466582,1.537388,0.428377


In [9]:
cp.cp

mapie.classification.MapieClassifier

In [7]:
pred = cp.cp.predict(X=X_test,alpha=0.05)

TypeError: MapieClassifier.predict() missing 1 required positional argument: 'self'

In [3]:
from IPython.display import display
model = ModelDeveloper(activity_col="Activity", id_col="ID", select_model="KNeighborsClassifier", n_jobs=-1)
model.fit(train_data)
model.classes_
model.predict(test_data)

,ID,Predicted values,Probability for class 0,Probability for class 1
0,0,0,0.55,0.45
1,1,1,0.45,0.55
2,2,1,0.35,0.65
3,3,1,0.45,0.55
4,4,1,0.45,0.55
5,5,1,0.45,0.55
6,6,1,0.45,0.55
7,7,0,0.55,0.45
8,8,1,0.45,0.55
9,9,1,0.45,0.55


In [4]:
# class: calibrate(X, y, oob=False, mc=None, seed=None, class_cond=False, nc=<function hinge>)
# reggg: calibrate(X, y, oob=False, mc=None, seed=None, cps=False, de=None)

In [5]:
unc = ConformalPredictor(model=model, activity_col="Activity", id_col="ID")
unc.calibrate(data=cal_data)
unc.predict(data=cal_data, confidence=0.99)

,ID,Predicted set,P-value for class 0,P-value for class 1,Actual values
0,0,"0, 1",0.228828,0.707947,0
1,1,"0, 1",0.385039,0.753084,1
2,2,"0, 1",0.680230,0.221313,0
3,3,"0, 1",0.156296,0.883580,0
4,4,"0, 1",0.208325,0.879413,0
5,5,"0, 1",0.641932,0.581625,1
6,6,"0, 1",0.282970,0.822299,0
7,7,"0, 1",0.129171,0.818456,1
8,8,"0, 1",0.131710,0.923976,0
9,9,"0, 1",0.242497,0.764085,1


In [6]:
unc.predict(data=test_data, confidence=0.99)

,ID,Predicted set,P-value for class 0,P-value for class 1,Actual values
0,0,"0, 1",0.743805,0.338942,1
1,1,"0, 1",0.361586,0.752536,0
2,2,"0, 1",0.018746,0.971922,0
3,3,"0, 1",0.293210,0.813037,0
4,4,"0, 1",0.292752,0.717048,1
5,5,"0, 1",0.394068,0.757615,1
6,6,"0, 1",0.258620,0.687545,1
7,7,"0, 1",0.772576,0.356760,1
8,8,"0, 1",0.260157,0.747798,1
9,9,"0, 1",0.371827,0.701951,0


In [7]:
result = unc.evaluate(cal_data, confidence=[0.9, 0.95, 0.99])
display(result)

,0.90,0.95,0.99
error,0.10000,0.025000,0.00000
avg_c,1.87500,1.950000,2.00000
one_c,0.12500,0.050000,0.00000
empty,0.00000,0.000000,0.00000
time_fit,0.00001,0.000010,0.00001
time_evaluate,0.00223,0.001753,0.00157


## REGRESSION

In [8]:
reg_train_data = create_regression_data(random_state=42)
reg_cal_data = create_regression_data(random_state=41)
reg_test_data = create_regression_data(random_state=40)

model_reg = ModelDeveloper(activity_col="Activity", id_col="ID", select_model="KNeighborsRegressor", n_jobs=-1)
model_reg.fit(reg_train_data)
model_reg.predict(test_data)

,ID,Predicted values
0,0,-6.161565
1,1,-8.379141
2,2,44.879611
3,3,23.483242
4,4,115.076877
5,5,-84.124758
6,6,84.661054
7,7,38.251261
8,8,-32.792953
9,9,-5.912005


In [9]:
unc_reg = ConformalPredictor(model=model_reg,activity_col="Activity", id_col="ID")
unc_reg.calibrate(data=reg_cal_data)
unc_reg.predict(data=reg_cal_data)

,ID,Predicted values,Lower Bound,Upper Bound,Actual values
0,0,7.518468,-318.074870,333.111806,365.899172
1,1,-77.732990,-403.326328,247.860348,61.207526
2,2,49.329340,-276.263998,374.922677,229.617185
3,3,-89.221669,-414.815007,236.371669,-42.182753
4,4,77.442364,-248.150974,403.035702,-194.474185
5,5,76.388893,-249.204444,401.982231,29.149098
6,6,-26.232642,-351.825980,299.360696,-76.713444
7,7,16.687456,-308.905881,342.280794,-6.300805
8,8,45.659792,-279.933546,371.253130,-62.648477
9,9,18.995789,-306.597549,344.589126,-251.304905


In [10]:
unc_reg.predict(data=reg_test_data, confidence=0.99)

,ID,Predicted values,Lower Bound,Upper Bound,Actual values
0,0,-27.988923,-353.582261,297.604415,-54.166874
1,1,133.896230,-191.697108,459.489568,66.713880
2,2,108.093713,-217.499625,433.687051,-223.311602
3,3,30.488192,-295.105146,356.081530,-438.919188
4,4,40.711690,-284.881647,366.305028,-73.137329
5,5,2.298347,-323.294991,327.891684,321.578936
6,6,-48.555567,-374.148904,277.037771,164.945827
7,7,-69.564176,-395.157514,256.029162,321.462077
8,8,45.010447,-280.582891,370.603785,-1.291498
9,9,76.955709,-248.637629,402.549047,379.673690


In [11]:
reg_test_data.head(5)

,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,Feature10,...,Feature33,Feature34,Feature35,Feature36,Feature37,Feature38,Feature39,Feature40,ID,Activity
0,-0.352509,0.603596,0.888519,-0.379498,0.456003,0.231560,0.224634,0.768409,0.573097,-0.186394,...,-1.517027,-0.733490,0.370729,-1.218588,0.303954,3.437449,-0.729998,-0.212551,0,-54.166874
1,-0.638448,0.106805,0.579245,-1.469121,-0.463197,0.696390,0.219730,1.435638,0.727683,-1.266901,...,0.882372,-2.479989,2.750117,0.470592,-0.931520,-2.253930,-1.267910,-0.844501,1,66.713880
2,0.127791,0.160145,0.201655,-0.305343,-0.553895,-0.018380,0.212288,0.915757,-0.020053,-0.293106,...,0.302212,0.218784,-0.982779,1.711772,-0.170051,1.551404,-0.317209,-1.092495,2,-223.311602
3,-1.730417,0.309116,-0.277753,-0.325738,0.092337,-0.169391,0.967418,0.513416,-2.183757,-1.061362,...,0.743496,-0.892318,0.575248,0.207556,1.396478,-0.769391,-1.124178,-2.178363,3,-438.919188
4,-1.168060,0.769628,1.142491,-0.383104,0.008148,-1.213530,-0.272442,-0.051291,-0.952321,-1.133724,...,0.135771,0.517249,-1.785718,-0.060118,0.577909,-0.478411,-1.144777,-0.494021,4,-73.137329


In [12]:
unc_reg.evaluate(reg_cal_data, confidence=[0.9, 0.95, 0.99])

/home/labhhc4/anaconda3/envs/proqsar/lib/python3.11/site-packages/crepes/base.py:578: UserWarning: the no. of calibration examples is too smallfor the chosen confidence level; the intervals will be of maximum size
  warnings.warn("the no. of calibration examples is too small" \


,0.90,0.95,0.99
error,0.075000,0.025000,0.000000
eff_mean,543.833098,651.186676,inf
eff_med,543.833098,651.186676,inf
time_fit,0.000008,0.000008,0.000008
time_evaluate,0.000113,0.000064,0.000416


In [13]:
no_y_data = reg_test_data.drop(columns='Activity', inplace=False)
no_y_data.head(5)

,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,Feature10,...,Feature32,Feature33,Feature34,Feature35,Feature36,Feature37,Feature38,Feature39,Feature40,ID
0,-0.352509,0.603596,0.888519,-0.379498,0.456003,0.231560,0.224634,0.768409,0.573097,-0.186394,...,-1.950069,-1.517027,-0.733490,0.370729,-1.218588,0.303954,3.437449,-0.729998,-0.212551,0
1,-0.638448,0.106805,0.579245,-1.469121,-0.463197,0.696390,0.219730,1.435638,0.727683,-1.266901,...,1.274852,0.882372,-2.479989,2.750117,0.470592,-0.931520,-2.253930,-1.267910,-0.844501,1
2,0.127791,0.160145,0.201655,-0.305343,-0.553895,-0.018380,0.212288,0.915757,-0.020053,-0.293106,...,0.488229,0.302212,0.218784,-0.982779,1.711772,-0.170051,1.551404,-0.317209,-1.092495,2
3,-1.730417,0.309116,-0.277753,-0.325738,0.092337,-0.169391,0.967418,0.513416,-2.183757,-1.061362,...,-1.363525,0.743496,-0.892318,0.575248,0.207556,1.396478,-0.769391,-1.124178,-2.178363,3
4,-1.168060,0.769628,1.142491,-0.383104,0.008148,-1.213530,-0.272442,-0.051291,-0.952321,-1.133724,...,-0.542065,0.135771,0.517249,-1.785718,-0.060118,0.577909,-0.478411,-1.144777,-0.494021,4


In [14]:
unc_reg.evaluate(no_y_data, confidence=0.95)

KeyError: "'Activity' column is not found in the provided data. Please ensure that the data contains this column in order to use this function."

In [15]:
from IPython.display import display
from ProQSAR.ApplicabilityDomain.applicability_domain import ApplicabilityDomain

In [16]:
train_data.head(5)

,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,Feature10,...,Feature33,Feature34,Feature35,Feature36,Feature37,Feature38,Feature39,Feature40,ID,Activity
0,1.053363,-0.186971,-1.065114,1.989145,0.028181,-1.311836,-0.755745,7.079563,-0.290275,0.010353,...,0.919076,-0.009119,0.817766,-1.161099,-2.521807,-0.231860,0.486502,0.536510,0,0
1,-1.302321,0.700310,0.493318,0.359700,0.186767,2.632382,0.026091,0.093234,0.982691,-0.949399,...,0.256030,-0.755383,-1.351685,-0.862925,3.193599,1.991874,-0.096060,0.517659,1,0
2,0.493151,-1.143005,1.105526,1.571264,-0.190682,1.050654,-1.053416,2.207330,-2.896255,0.201099,...,0.998010,0.217433,0.364961,0.070504,-0.841747,-1.963083,-0.754276,-0.623769,2,1
3,3.062493,1.117296,-1.251539,2.971553,0.650201,0.352055,-0.564079,-0.479588,-2.198806,-0.241236,...,-1.448014,-0.099176,-0.691908,0.637222,-0.691468,0.004744,0.447709,-1.280304,3,0
4,1.682497,1.160827,-0.362441,3.071084,-0.550305,-1.739714,0.395804,2.627580,0.883660,1.148766,...,-0.172627,-0.671623,-0.370614,-2.900122,-1.124807,-0.894226,0.477041,0.494030,4,1


In [19]:
ad = ApplicabilityDomain(method='lof', activity_col='Activity', id_col='ID')
ad.fit(train_data)

In [21]:
ad_values = ad.predict(train_data)
ad_values

,Applicability domain,ID
0,in,0
1,in,1
2,in,2
3,in,3
4,in,4
5,in,5
6,in,6
7,in,7
8,in,8
9,in,9


,Applicability domain
0,in
1,in
2,in
3,in
4,in
5,in
6,in
7,in
8,in
9,in


In [ ]:
ad.predict(X_test)

array([-0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5,
       -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5,
       -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5,
       -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5])

In [ ]:
ad.predict(X_test)

array([0.07985261, 0.09524855, 0.02568171, 0.12721055, 0.15106405,
       0.15352611, 0.16804356, 0.169854  , 0.16869537, 0.14451552,
       0.16094203, 0.19649002, 0.19701688, 0.22309988, 0.21187386,
       0.23292453, 0.22524326, 0.24017089, 0.23771524, 0.2409936 ,
       0.2375617 , 0.24323891, 0.19277318, 0.23455938, 0.22275886,
       0.21767605, 0.21336508, 0.12873042, 0.16447175, 0.2007852 ,
       0.17340135, 0.14967104, 0.15285767, 0.14865575, 0.15615271,
       0.13664181, 0.03366138, 0.12102857, 0.03204309, 0.09346139])